In [1]:
import scanpy as sc 
import squidpy as sq
from CASP import CASP
import numpy as np

adata = sc.read_h5ad('keren1.h5ad')
casp = CASP(adata)
casp.causal_discovery(show=False)
leaf_nodes, leaf_index = casp.get_leaf_nodes()

train_index = np.ones(adata.X.shape[1]).astype(bool)
train_index[leaf_index] = 0

X_total = adata.X

X =  X_total[:,train_index]

y = X_total[:,~train_index]

/data/civy/miniconda3/envs/causal/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
/data/civy/miniconda3/envs/causal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detecting 3 CUDA device(s).


Method : LiNGAM


In [ ]:
import numpy as np

X_total = adata.X
np.random.seed(42)
random_index = np.random.choice(np.arange(X_total.shape[1]), len(leaf_nodes), replace=False)
train_index = np.ones(adata.X.shape[1]).astype(bool)
train_index[random_index] = 0

X =  X_total[:,train_index]

y = X_total[:,~train_index]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(torch.from_numpy(X).float(), torch.from_numpy(y).float(), test_size=0.2, random_state=42)

# Create TensorDatasets for train and test sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the MLP model for regression
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(X.shape[1], 128) 
        self.layer2 = nn.Linear(128, 64)  
        #self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(64, y.shape[1])
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.dropout(torch.relu(self.layer1(x)))
        x = self.dropout(torch.relu(self.layer2(x)))
        #x = self.dropout(torch.relu(self.layer3(x)))
        x = self.output(x)
        return x

# Initialize the model, optimizer and loss function
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.MSELoss()

# Training loop
def train(model, train_loader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

# Training the model
for epoch in range(1000):  # number of epochs
    train(model, train_loader, optimizer, criterion)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            predictions.extend(output.view(-1).tolist())
            targets.extend(y_batch.view(-1).tolist())
    return r2_score(targets, predictions)

r2 = evaluate(model, test_loader)
print(f"Test R^2 Score: {r2}")
